Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
num_steps = 3000
batch_size = 125
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    w2 = tf.Variable(tf.truncated_normal([hidden_nodes,num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    x_train = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    x_valid = tf.constant(valid_dataset)
    y_valid = tf.constant(valid_labels)
    x_test = tf.constant(test_dataset)
    y_test = tf.constant(test_labels)

    logits = tf.matmul(tf.nn.relu(tf.matmul(x_train,w1)+b1),w2)+b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train,logits=logits)) + 0.003*tf.nn.l2_loss(w1) + 0.003*tf.nn.l2_loss(w2)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_train,w1)+b1),w2)+b2)
    valid_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_valid,w1)+b1),w2)+b2)
    test_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_test,w1)+b1),w2)+b2)
    
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())
    for i in range(num_steps):
        offset = i * batch_size % (train_dataset.shape[0] - batch_size)
        feed_dict = {
            x_train: train_dataset[offset:offset+batch_size], 
            y_train: train_labels[offset:offset+batch_size]
        }
        _,l,predictions = session.run([optimizer,loss,train_predictions], feed_dict=feed_dict)
        if (i+1)%100 == 0:
            print("training loss:", l)
            print("training accuracy: ",accuracy(predictions, train_labels[offset:offset+batch_size]))
            print("valid accuracy: ",accuracy(session.run(valid_predictions), valid_labels))
            print("-" *20)
    print("test accuracy: ",accuracy(session.run(test_predictions), test_labels))


training loss: 714.405
training accuracy:  84.0
valid accuracy:  78.17
--------------------
training loss: 526.198
training accuracy:  76.0
valid accuracy:  77.8
--------------------
training loss: 390.716
training accuracy:  76.8
valid accuracy:  71.78
--------------------
training loss: 290.016
training accuracy:  77.6
valid accuracy:  79.32
--------------------
training loss: 208.641
training accuracy:  85.6
valid accuracy:  79.95
--------------------
training loss: 154.84
training accuracy:  83.2
valid accuracy:  80.03
--------------------
training loss: 113.813
training accuracy:  83.2
valid accuracy:  81.62
--------------------
training loss: 84.0146
training accuracy:  80.0
valid accuracy:  83.36
--------------------
training loss: 62.3657
training accuracy:  82.4
valid accuracy:  83.46
--------------------
training loss: 46.2253
training accuracy:  82.4
valid accuracy:  84.72
--------------------
training loss: 34.2645
training accuracy:  84.0
valid accuracy:  84.91
-----------

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [6]:
num_steps = 3000
batch_size = 125
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    w2 = tf.Variable(tf.truncated_normal([hidden_nodes,num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    x_train = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    x_valid = tf.constant(valid_dataset)
    y_valid = tf.constant(valid_labels)
    x_test = tf.constant(test_dataset)
    y_test = tf.constant(test_labels)

    logits = tf.matmul(tf.nn.relu(tf.matmul(x_train,w1)+b1),w2)+b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train,logits=logits)) + 0.003*tf.nn.l2_loss(w1) + 0.003*tf.nn.l2_loss(w2)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_train,w1)+b1),w2)+b2)
    valid_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_valid,w1)+b1),w2)+b2)
    test_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_test,w1)+b1),w2)+b2)
    
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())
    for i in range(num_steps):
        offset = i * batch_size % (batch_size*4- batch_size)
        feed_dict = {
            x_train: train_dataset[offset:offset+batch_size], 
            y_train: train_labels[offset:offset+batch_size]
        }
        _,l,predictions = session.run([optimizer,loss,train_predictions], feed_dict=feed_dict)
        if (i+1)%100 == 0:
            print("training loss:", l)
            print("training accuracy: ",accuracy(predictions, train_labels[offset:offset+batch_size]))
            print("valid accuracy: ",accuracy(session.run(valid_predictions), valid_labels))
            print("-" *20)
    print("test accuracy: ",accuracy(session.run(test_predictions), test_labels))


training loss: 701.336
training accuracy:  100.0
valid accuracy:  74.38
--------------------
training loss: 519.445
training accuracy:  100.0
valid accuracy:  74.36
--------------------
training loss: 384.728
training accuracy:  100.0
valid accuracy:  74.33
--------------------
training loss: 284.949
training accuracy:  100.0
valid accuracy:  74.37
--------------------
training loss: 211.048
training accuracy:  100.0
valid accuracy:  74.42
--------------------
training loss: 156.313
training accuracy:  100.0
valid accuracy:  74.4
--------------------
training loss: 115.774
training accuracy:  100.0
valid accuracy:  74.28
--------------------
training loss: 85.7481
training accuracy:  100.0
valid accuracy:  74.26
--------------------
training loss: 63.5098
training accuracy:  100.0
valid accuracy:  74.25
--------------------
training loss: 47.0392
training accuracy:  100.0
valid accuracy:  74.19
--------------------
training loss: 34.8411
training accuracy:  100.0
valid accuracy:  74.36

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [7]:
num_steps = 3000
batch_size = 125
hidden_nodes = 1024
dropout = 0.5

graph = tf.Graph()
with graph.as_default():
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    w2 = tf.Variable(tf.truncated_normal([hidden_nodes,num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    x_train = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    x_valid = tf.constant(valid_dataset)
    y_valid = tf.constant(valid_labels)
    x_test = tf.constant(test_dataset)
    y_test = tf.constant(test_labels)

    logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(x_train,w1)+b1),dropout),w2)+b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train,logits=logits)) + 0.003*tf.nn.l2_loss(w1) + 0.003*tf.nn.l2_loss(w2)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_train,w1)+b1),w2)+b2)
    valid_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_valid,w1)+b1),w2)+b2)
    test_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_test,w1)+b1),w2)+b2)
    
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())
    for i in range(num_steps):
        offset = i * batch_size % (batch_size*5 - batch_size)
        feed_dict = {
            x_train: train_dataset[offset:offset+batch_size], 
            y_train: train_labels[offset:offset+batch_size]
        }
        _,l,predictions = session.run([optimizer,loss,train_predictions], feed_dict=feed_dict)
        if (i+1)%100 == 0:
            print("training loss:", l)
            print("training accuracy: ",accuracy(predictions, train_labels[offset:offset+batch_size]))
            print("valid accuracy: ",accuracy(session.run(valid_predictions), valid_labels))
            print("-" *20)
    print("test accuracy: ",accuracy(session.run(test_predictions), test_labels))

training loss: 705.454
training accuracy:  100.0
valid accuracy:  76.29
--------------------
training loss: 522.466
training accuracy:  100.0
valid accuracy:  77.14
--------------------
training loss: 387.044
training accuracy:  100.0
valid accuracy:  76.52
--------------------
training loss: 286.728
training accuracy:  100.0
valid accuracy:  77.04
--------------------
training loss: 212.394
training accuracy:  100.0
valid accuracy:  77.42
--------------------
training loss: 157.327
training accuracy:  100.0
valid accuracy:  77.03
--------------------
training loss: 116.534
training accuracy:  100.0
valid accuracy:  76.51
--------------------
training loss: 86.3156
training accuracy:  100.0
valid accuracy:  76.74
--------------------
training loss: 63.9358
training accuracy:  100.0
valid accuracy:  77.26
--------------------
training loss: 47.3575
training accuracy:  100.0
valid accuracy:  77.18
--------------------
training loss: 35.083
training accuracy:  100.0
valid accuracy:  77.25

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [55]:
num_steps = 3000
batch_size = 250
hidden_nodes = 1024
regularization_constant = 0.003
dropout_ratio = 1
starter_learning_rate = 0.4


graph = tf.Graph()
with graph.as_default():
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    w2 = tf.Variable(tf.truncated_normal([hidden_nodes,num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    x_train = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    x_valid = tf.constant(valid_dataset)
    y_valid = tf.constant(valid_labels)
    x_test = tf.constant(test_dataset)
    y_test = tf.constant(test_labels)
    
    h1 = tf.nn.relu(tf.matmul(x_train,w1)+b1)
    logits = tf.matmul(tf.nn.dropout(h1,dropout_ratio),w2)+b2
    
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.98, staircase=False)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train,logits=logits)) 
    loss +=  regularization_constant * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    train_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_train,w1)+b1),w2)+b2)
    valid_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_valid,w1)+b1),w2)+b2)
    test_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(x_test,w1)+b1),w2)+b2)
    
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())
    print('Initialized')
    print('-'*60)
    print('{:^12s}|{:^25s}|{:^12s}'.format("","Minibatch","Validation"))
    print('{:^12s}|{:^12s}|{:^12s}|{:^12s}|{:^12s}'.format("Step","Loss","Accuracy","Accuracy","Alpha"))
    for i in range(num_steps):
        offset = i * batch_size % (train_dataset.shape[0] - batch_size)
        feed_dict = {
            x_train: train_dataset[offset:offset+batch_size], 
            y_train: train_labels[offset:offset+batch_size]
        }
        _,l,predictions = session.run([optimizer,loss,train_predictions], feed_dict=feed_dict)
        if (i+1)%100 == 0:
            print('{:^12d}|{:^12.2f}|{:^12.2f}|{:^12.2f}|{:^12.2f}'
                  .format(i+1,
                          l,
                          accuracy(predictions, train_labels[offset:offset+batch_size]),
                          accuracy(valid_predictions.eval(), valid_labels), 
                          learning_rate.eval()))
    print('-'*60)
    print("test accuracy: ",accuracy(session.run(test_predictions), test_labels))

Initialized
------------------------------------------------------------
            |        Minibatch        | Validation 
    Step    |    Loss    |  Accuracy  |  Accuracy  |   Alpha    
    100     |   761.86   |   80.80    |   78.41    |    0.39    
    200     |   595.45   |   80.00    |   78.90    |    0.38    
    300     |   470.99   |   80.40    |   79.21    |    0.38    
    400     |   374.12   |   81.20    |   81.48    |    0.37    
    500     |   298.93   |   86.00    |   81.91    |    0.36    
    600     |   240.26   |   82.80    |   82.84    |    0.35    
    700     |   194.67   |   79.60    |   78.21    |    0.35    
    800     |   157.23   |   89.60    |   83.37    |    0.34    
    900     |   128.61   |   84.80    |   83.61    |    0.33    
    1000    |   105.37   |   86.00    |   82.77    |    0.33    
    1100    |   86.89    |   84.00    |   84.37    |    0.32    
    1200    |   71.73    |   86.00    |   85.03    |    0.31    
    1300    |   59.66    |   8